# Tensorflow 2.x + TensorBoard
## 引入依赖

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2022-11-15 15:46:54.815531: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Num GPUs Available:  2


2022-11-15 15:46:56.212338: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-15 15:46:56.213721: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-15 15:46:56.248904: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6325GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-11-15 15:46:56.249986: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:82:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6325GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-11-15 15:46:56.250021: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
202

# 定义参数

In [2]:
# Training parameters.
learning_rate = 0.01
training_steps = 2000
batch_size = 256
display_step = 100


# 数据集

In [3]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255., x_test / 255.
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# 模型定义

In [4]:
class SimpleConvNet(Model):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = layers.Conv2D(32, 3, activation='relu')
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(128, activation='relu')
        self.d2 = layers.Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

neural_net = SimpleConvNet()
help(neural_net.fit)

2022-11-15 15:46:57.293142: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-15 15:46:57.557548: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6325GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-11-15 15:46:57.558474: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:82:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.6325GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-11-15 15:46:57.558517: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-11-15 15:46:57.558545: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11


Help on method fit in module tensorflow.python.keras.engine.training:

fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False) method of __main__.SimpleConvNet instance
    Trains the model for a fixed number of epochs (iterations on a dataset).
    
    Arguments:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return a tuple
            of either

# 训练和验证

In [5]:
neural_net.compile(tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')])
import datetime
log_dir = "../logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

neural_net.fit(x=x_train, 
          y=y_train, 
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback],
          batch_size=64
          )

2022-11-15 15:46:58.843074: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-11-15 15:46:58.843132: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-11-15 15:46:58.843181: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 2 GPUs
2022-11-15 15:46:58.843468: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcupti.so.11.0'; dlerror: libcupti.so.11.0: cannot open shared object file: No such file or directory
2022-11-15 15:46:58.845299: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so
2022-11-15 15:46:59.324379: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-11-15 15:46:59.324622: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed
2022-11-15 15:46:59.630318: I tensorflow/compiler/mlir/mlir_gra

Epoch 1/10


2022-11-15 15:47:00.024150: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-11-15 15:47:00.389006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-11-15 15:47:00.391928: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-11-15 15:47:01.785493: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-11-15 15:47:01.901401: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


  3/938 [..............................] - ETA: 11:00 - loss: 2.9999 - train_accuracy: 0.1562 

2022-11-15 15:47:04.978638: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-11-15 15:47:04.978704: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-11-15 15:47:06.207316: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-11-15 15:47:06.207802: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed
2022-11-15 15:47:06.331876: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 87 callback api events and 75 activity events. 
2022-11-15 15:47:06.338519: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-11-15 15:47:06.345231: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ../logs/20221115-154658/train/plugins/profile/2022_11_15_15_47_06
2022-11-15 15:47:06.349432: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] D

938/938 [==============================] - 16s 12ms/step - loss: 0.3002 - train_accuracy: 0.9150 - val_loss: 0.0853 - val_train_accuracy: 0.9723
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0558 - train_accuracy: 0.9828 - val_loss: 0.0689 - val_train_accuracy: 0.9785
Epoch 3/10
938/938 [==============================] - 2s 3ms/step - loss: 0.0352 - train_accuracy: 0.9892 - val_loss: 0.0988 - val_train_accuracy: 0.9753
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0236 - train_accuracy: 0.9929 - val_loss: 0.1560 - val_train_accuracy: 0.9748
Epoch 5/10
938/938 [==============================] - 2s 3ms/step - loss: 0.0301 - train_accuracy: 0.9918 - val_loss: 0.1504 - val_train_accuracy: 0.9733
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0304 - train_accuracy: 0.9917 - val_loss: 0.1778 - val_train_accuracy: 0.9723
Epoch 7/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0227 - tr